## Notebook for Text Cluster Analysis
Queue from screen-analisis-ai where :
- status = 1
- jenis-analisa = 10
- order by created desc, limit 1

### Input as proces started
Record header and parameter information

 * Kesepakatan status di kolom screen_analisis_ai.status
 * 1 --> baru diinput
 * 2 --> lagi dikerjakan
 * 3 --> proses berhasil
 * 4 --> proses gagal
 

In [1]:
import pymysql.cursors
import pandas as pd
import uuid
import time
import os
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

from sqlalchemy import create_engine, Column, Integer, String, MetaData, Table
from sqlalchemy.sql import text
from sqlalchemy.orm import sessionmaker
import pandas as pd

# import custom data connector
import data_connector



In [2]:
strSQL = """
select 	a.id as jobsid,
        a.*,
		d.id as key_monitoring_media_social, 
		e.id as key_monitoring_media_online,
		c.*
from screen_analisis_ai a inner join monitoring b
	on a.monitoring_id = cast(b.id as varchar)
	inner join monitoring_search c 
		on b.id = c.monitoring_id 
	inner join monitoring_media_social d
		on d.monitoring_id = c.monitoring_id
	left outer join monitoring_media_online e
		on e.monitoring_id = c.monitoring_id 
where a.jenis_analisa = '10'
and a.status = 1
order by a.created desc 
limit 1
"""

df_job = data_connector.execute_query_psql(strSQL)
if len(df_job) == 0:
    # get out, nothing to do
    print('Zero jobs, quitting now')
    quit()
    
similarity_treshold = 0.9
i_process_id = df_job['jobsid'][0]
screen_name = ''
database_keyword_id = df_job['key_monitoring_media_social'][0]
social_media_monitoring_id = df_job['key_monitoring_media_social'][0]
media_online_monitoring_id = df_job['key_monitoring_media_online'][0]

# print(database_keyword_id)
print(similarity_treshold)
print(i_process_id)
print(social_media_monitoring_id)
print(media_online_monitoring_id)


# Prepare SQL Statement
print(i_process_id)
sql = "update screen_analisis_ai set status = 2, last_status_update = now(), start_process = now() where id = %s"
sql = sql.replace('%s', str(i_process_id))

print(sql)
row_count = data_connector.execute_query_psql(sql)
print('update ' + str(row_count) + ' rows')


0.9
3935
eb784a56-6fcb-4252-b0b8-21402b8c286f
115fd4ed-8608-4137-8170-e8d49238dece
3935
update screen_analisis_ai set status = 2, last_status_update = now(), start_process = now() where id = 3935
update 1 rows


In [4]:
# marking jobs
# Prepare SQL Statement
print(i_process_id)
sql = "update screen_analisis_ai set status = 2, last_status_update = now(), start_process = now() where id = %s"
sql = sql.replace('%s', str(i_process_id))

print(sql)
row_count = data_connector.execute_query_psql(sql)
print('update ' + str(row_count) + ' rows')

#
# Create Header Record
sql = "insert into ret_analysis_header (job_id, datetime_start, user_id) values (" + str(i_process_id) + ", now(), '1')"
# Execute the query
print(sql)
row_count = data_connector.execute_query_psql(sql)
#
# Create Parameter Record
sql = """
insert into ret_analysis_parameter 
(job_id, param_id, param_name, param_value) 
values (%s, %s, %s, %s)
""" % (str(i_process_id), '1', "'Similarity Treshold'", str(similarity_treshold))
# Execute the query
print(sql)
row_count = data_connector.execute_query_psql(sql)

sql = """
insert into ret_analysis_parameter 
(job_id, param_id, param_name, param_value) 
values (%s, %s, %s, '%s')
""" % (str(i_process_id), '1', "'DB_ID'", str(database_keyword_id))
print(sql)
row_count = data_connector.execute_query_psql(sql)



3935
update screen_analisis_ai set status = 2, last_status_update = now(), start_process = now() where id = 3935
update 1 rows
insert into ret_analysis_header (job_id, datetime_start, user_id) values (3935, now(), '1')

insert into ret_analysis_parameter 
(job_id, param_id, param_name, param_value) 
values (3935, 1, 'Similarity Treshold', 0.9)


insert into ret_analysis_parameter 
(job_id, param_id, param_name, param_value) 
values (3935, 1, 'DB_ID', 'eb784a56-6fcb-4252-b0b8-21402b8c286f')



In [5]:
import string
import re
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import array as arr
from tqdm import tqdm

# Text Preprocessing, 
def text_preproc(strIn):
    # case folding
    strOut = strIn.lower()
    # remove numbers
    strOut = re.sub(r"\d+", "", strOut)
    # remote punctuation
    strOut = strOut.translate(str.maketrans("","",string.punctuation))
    # remove whitspace
    strOut = strOut.strip()
    # 
    strOut = re.sub('\s+',' ',strOut)
    
    return strOut
# end Text Preprocessing

def calculate_similarity(X_set, Y_set):
# Program to measure the similarity between  
# two sentences using cosine similarity. 

    l1 =[];l2 =[]

    # form a set containing keywords of both strings  
    rvector = X_set.union(Y_set)  
    for w in rvector: 
        if w in X_set: l1.append(1) # create a vector
        else: l1.append(0) 
        if w in Y_set: l2.append(1) 
        else: l2.append(0) 
    c = 0

    # cosine formula  
    for i in range(len(rvector)): 
            c+= l1[i]*l2[i]
    try:
        cosine = c / float((sum(l1)*sum(l2))**0.5) 
    except:
        # print('zero div on X_set')
        return 0;
        
    return cosine

def largest_in_col(arr,nCol):
    # 
    # Find largest value of col nCol on 2D arr
    #
    
    # init value
    max_val = arr[0][nCol]
    # also, remember index
    row_index = 0
    
    for x in range(0, len(arr)):
        if arr[x][nCol] > max_val:
            max_val = arr[x][nCol]
            row_index = x
        
    return max_val,row_index

def smallest_in_col(arr,nCol):
    # 
    # Find smallest value of col nCol on 2D arr
    #
    
    # init value
    min_val = arr[0][nCol]
    # also, remember index
    row_index = 0
    
    for x in range(0, len(arr)):
        if arr[x][nCol] < min_val:
            min_val = arr[x][nCol]
            row_index = x
        
    return min_val,row_index

def process_text_content(df, platform_id, i_process_id):
    # Apply Text Preproc
    df['text_content'] = df['text_content'].apply(text_preproc)
    
    # Apply to data frame
    df['text_content'] = df['text_content'].apply(text_preproc)
    df.head()
    
    from nltk.corpus import stopwords 
    
    # Load stopwords
    sw = stopwords.words('indonesian')
    
    # apply tokenize
    df['tokenized_tweet'] = df.apply(lambda row: nltk.word_tokenize(row['text_content']), axis=1)
    
    # apply stopword removal
    df['sw'] = df.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)
    
    df.head()
    
    
    st = similarity_treshold
    cluster_no = 1
    s_score = 0
    s_score_current = 0
    i_current_cluster = 0
    
    #create zero element array
    #col 0 => index base tweet
    #col 1 => cluster number
    #col 2 => similarity score
    base_tweet = []
    
    #proceed to compare to all tweet
    cosine_matrix = np.zeros(( len(df), len(df) ), dtype=np.dtype('f4'))
    
    # flag the mt
    with tqdm(total=( (len(df)*len(df)/2))-(len(df)/2))  as pbar:
        for j in range(0, len(df)):
            tweet_to_compare = df['sw'][j]
    
            #check, is this second tweet?
            if(j == 0):
                #first tweet, add as cluster no #1
                base_tweet.append([j,1,1.0])
                i_current_cluster = base_tweet[0][1]
    
            elif(j == 1):
                #compare to prev tweet
                s_score = calculate_similarity(tweet_to_compare, df['sw'][ base_tweet[0][0] ])
    
                if(s_score < st):
                    #not similar
                    base_tweet.append([j,2,1])
                    i_current_cluster = base_tweet[j][1]
    
            else:
                #other else tweet
                for x in range(0,len(base_tweet)):
                    #compare every element
                    s_score = calculate_similarity(tweet_to_compare, df['sw'][ base_tweet[x][0] ])
                    base_tweet[x][2] = s_score
    
                if(largest_in_col(base_tweet,2)[0] < st):
                    #no similar, add as one new cluster
                    i_current_cluster = i_current_cluster + 1
                    base_tweet.append([j,i_current_cluster,largest_in_col(base_tweet,2)[0]])
                else:
                    #determine cluster# from biggest similarity
                    i_current_cluster = base_tweet[(largest_in_col(base_tweet,2)[1])][1]
    
            #proceed to compare to all tweet
            for i in range(0,len(df)):
                # update progress
                if (j<i):
                    pbar.update(1)
                    s_score = calculate_similarity(tweet_to_compare, df['sw'][i])
                    if (s_score >= st):
                        cosine_matrix[i,j] = i_current_cluster
    
    
    pbar.close()
    # print(cosine_matrix)
    #
    # Create Tweet Cluster Record
    sql = "insert into ret_cluster_result_monitor (ref_id, cluster_no, platform_id, job_id) values ('%s', %s, %s, %s)"
    
    #print(len(cosine_matrix))
    #print(i_process_id)
    
    # initiate cluster number
    i_cluster_no_save = 0
    temp_val = 0
                     
    ## inserting tweet cluster
    for i in range(0, len(cosine_matrix)):
        
        # find value in this particular row
        for j in range(0,len(cosine_matrix[i])):
            
            # print(cosine_matrix[i][j])
            temp_val = cosine_matrix[j][i]
            
            if(temp_val != 0):
                i_cluster_no_save = temp_val
    
        # Execute the query
        data_connector.execute_query_psql(sql % (df['ref_id'][i], i_cluster_no_save, platform_id, i_process_id,))
        # print(sql % (df['ref_id'][i], i_cluster_no_save, platform_id, i_process_id))
    
    print('finished inserting cluster data')
    
    


In [6]:
# 10 = tiktok
# 20 = youtube
# 30 = instagram_post
# 40 = facebook_post
# 50 = google_result

# Processing jobs for each platform
iRowCount = 0

if df_job['is_tiktok'][0]:
    sql = '''
    select id as ref_id, "desc" as text_content 
    from tiktok 
    where monitoring_id = '%s' -- 10
    '''
    df = data_connector.execute_query_psql(sql % (social_media_monitoring_id))
    process_text_content(df,10, i_process_id)
    iRowCount = iRowCount + len(df)

if df_job['is_youtube'][0]:
    sql = '''
    select 	id as ref_id, title as text_content
    from youtube 
    where monitoring_id = '%s' -- 20
    '''
    df = data_connector.execute_query_psql(sql % (social_media_monitoring_id))
    process_text_content(df,20, i_process_id)
    iRowCount = iRowCount + len(df)

if df_job['is_instagram'][0]:
    sql = '''
    select id as ref_id, content as text_content
    from instagram_post 
    where monitoring_id = '%s' -- 30
    '''
    df = data_connector.execute_query_psql(sql % (social_media_monitoring_id))
    process_text_content(df,30, i_process_id)
    iRowCount = iRowCount + len(df)

if df_job['is_facebook'][0]:
    sql = '''
    select id as ref_id, "description" as text_content
    from facebook_post 
    where monitoring_id = '%s' -- 40
    and length(trim("description")) > 0
    '''
    df = data_connector.execute_query_psql(sql % (social_media_monitoring_id))
    process_text_content(df,40, i_process_id)
    iRowCount = iRowCount + len(df)

# if row_count['is_google'][0]:
#     sql = '''
#     select 	id as ref_id, "description" as text_content
#     from google_result 
#     where monitoring_id = '%s' -- 50
#     '''
#     df_google = data_connector.execute_query_psql(sql % (social_media_monitoring_id))
#     process_text_content(df_google,50, i_process_id)


print("Total Processed Data: " + str(iRowCount))

100%|██████████████████████████████████████████████████████████████████████████████████| 12246/12246.0 [00:00<00:00, 42454.29it/s]


finished inserting cluster data


100%|████████████████████████████████████████████████████████████████████████████████| 142311/142311.0 [00:02<00:00, 56191.39it/s]


finished inserting cluster data


100%|██████████████████████████████████████████████████████████████████████████████| 3523185/3523185.0 [03:10<00:00, 18523.97it/s]


finished inserting cluster data


100%|████████████████████████████████████████████████████████████████████████████████| 175528/175528.0 [00:06<00:00, 25223.58it/s]


finished inserting cluster data
Total Processed Data: 3939


In [ ]:
# Finishing Jobs
# Create Parameter Record
sql = "insert into ret_analysis_parameter (job_id, param_id, param_name, param_value) values (%s, %s, %s, %s)"
# Execute the query
data_connector.execute_query_psql(sql % (i_process_id, 1, "'#Content Processed'",iRowCount))

# Create Tweet Cluster Record
sql = "update ret_analysis_header set datetime_finish = NOW() where job_id = %s"
# Executing query
data_connector.execute_query_psql(sql % (i_process_id) )

sql = "update screen_analisis_ai set status = 3, duration = EXTRACT(EPOCH FROM (now() - start_process)), end_process = NOW() where id = %s"
data_connector.execute_query_psql(sql % (i_process_id))

print('inserting result finished')



In [ ]:
# Wait 10 sec before release
time.sleep(10)